In [ ]:
#setup

import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

In [ ]:
#recombine data

from combine import recombine
data = recombine("../data/potholes_1.csv", "../data/potholes_2.csv")

In [ ]:
#extract days from date data by converting date to time, creating completion time variable, and turning to an int

data["creation_date"] = pd.to_datetime(data["CREATION DATE"])
data["completion_date"] = pd.to_datetime(data["COMPLETION DATE"])
data["creation_year"] = pd.DatetimeIndex(data["creation_date"]).year

data["creation_date"] = pd.to_datetime(data["creation_date"], format = "%m/%d/%Y")
data["completion_date"] = pd.to_datetime(data["completion_date"], format = "%m/%d/%Y")

data["completion_time"] = data["completion_date"] - data["creation_date"]

data["completion_time"] = data["completion_time"].apply(lambda x: x.days)

In [ ]:
#inefficiently clean data

data.drop('CREATION DATE', axis=1, inplace=True)
data.drop('COMPLETION DATE', axis=1, inplace=True)
data.drop('STATUS', axis=1, inplace=True)
data.drop('SERVICE REQUEST NUMBER', axis=1, inplace=True)
data.drop('TYPE OF SERVICE REQUEST', axis=1, inplace=True)
data.drop('CURRENT ACTIVITY', axis=1, inplace=True)
data.drop('MOST RECENT ACTION', axis=1, inplace=True)
data.drop('STREET ADDRESS', axis=1, inplace=True)
data.drop('X COORDINATE', axis=1, inplace=True)
data.drop('Y COORDINATE', axis=1, inplace=True)
data.drop('SSA', axis=1, inplace=True)
data.drop('LATITUDE', axis=1, inplace=True)
data.drop('LONGITUDE', axis=1, inplace=True)
data.drop('LOCATION', axis=1, inplace=True)
data.drop('creation_date', axis=1, inplace=True)
data.drop('completion_date', axis=1, inplace=True)
data.drop('ZIP', axis=1, inplace=True)
data.drop('Ward', axis=1, inplace=True)
data.drop('Police District', axis=1, inplace=True)
data.drop('NUMBER OF POTHOLES FILLED ON BLOCK', axis=1, inplace=True)
data = data.dropna()
data["community_area"] = data["Community Area"].astype(int)
data.drop('Community Area', axis=1, inplace=True)

In [ ]:
#data1 = data.groupby(["community_area"])["creation_year"].count()
#data1 = data1.drop(labels=0, axis=0)

#Y1 = data1["creation_year"]
#Y1.head

data1 = data.groupby(["community_area"])["creation_year"].count().reset_index(name="count")
data1 = data1.drop(labels=0, axis=0)
data1.head
Y1 = data1["count"]
Y1.reset_index(drop=True)


data2 = data.groupby(["community_area"]).agg({'completion_time': 'mean'})
data2 = data2.drop(labels=0, axis=0)
data2.head

In [ ]:
crime = pd.read_csv("../data/Crimes_data.csv")

crime["community_area"] = crime["Community Area"]
crimes = crime[['community_area', "ID"]]
crimes.dropna()

crimec = crimes.groupby(["community_area"]).count()
crimec.head


In [ ]:
income = pd.read_csv("../data/chicago_census_data.csv")

income = income.drop(labels=77, axis=0)
income["income"] = income["PER CAPITA INCOME "]
income["community_area"] = income["Community Area Number"].astype(int)
incomes = income[['community_area', "income"]]

incomes.head

In [ ]:
X = pd.merge(incomes, crimec, on = 'community_area')
X.drop('community_area', axis=1, inplace=True)
X.index += 1
X.head

In [ ]:
X2 = sm.add_constant(X)
est = sm.OLS(Y1, X2)
est2 = est.fit()
est2.summary()

In [ ]:
esty = sm.OLS(data2, X2)
est3 = esty.fit()
est3.summary()